In [1]:
import keras
import matplotlib.pyplot as plt
import numpy as np
keras.__version__

Using TensorFlow backend.


'2.1.5'

Apply IMDB data for illustration

In [2]:
from keras.datasets import imdb
from keras import preprocessing

# Number of words to consider as features
max_features = 10000
# Cut texts after this number of words 
# (among top max_features most common words)
maxlen = 500

# Load the data as lists of integers.
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

# This turns our lists of integers
# into a 2D integer tensor of shape `(samples, maxlen)`
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

In [3]:
%%bash
mkdir -p ./modelTrain

# Simple Dense model

In [ ]:
from keras import layers
from keras import Input
from keras.models import Model
from keras.layers import Flatten, Dense

Embed_dim=128
_input = Input(shape=(None,))

# We specify the maximum input length to our Embedding layer
# so we can later flatten the embedded inputs
x = layers.Embedding(max_features,Embed_dim, input_length=maxlen)(_input)
x = Flatten()(x)
predictions= Dense(1, activation='sigmoid')(x)


model = Model([_input], predictions)




In [ ]:


callbacks_list = [
    #Interrupts training when improvement stops
    #Monitors the model’s validation accuracy
    #Interrupts training when accuracy has stopped improving for more than one epoch (that is, two epochs)
        keras.callbacks.EarlyStopping(
            monitor='acc',
            patience=1,
        ),
    #Saves the current weights after every epoch Path to the destination model file
    #These two arguments mean you won’t overwrite the model file unless val_loss has improved, which allows you to keep the best model seen during training.
        keras.callbacks.ModelCheckpoint(
            filepath='./modelTrain/model7.2.h1',
            monitor='val_loss',
            save_best_only=True,
        )]
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
model.summary()


In [ ]:

history = model.fit(x_train, y_train,
                    epochs=40,
                    batch_size=32,
                    callbacks=callbacks_list,
                    validation_split=0.2)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

Add callback of jumping out of Plateau (taking smaller steps)<br>
Custom callback to save activation output of each layer

In [ ]:
import keras
import numpy as np
class ActivationLogger(keras.callbacks.Callback):
    #Called by the parent model before training, to inform the callback of what model will be calling it
    def set_model(self, model):
        self.model = model
        layer_outputs = [layer.output for layer in model.layers]
        print (np.shape(layer_outputs))
        self.activations_model = keras.models.Model(model.input,layer_outputs)
    def on_epoch_end(self, epoch, logs=None):
        if self.validation_data is None:
            raise RuntimeError('Requires validation_data.')
        validation_sample = self.validation_data[0][0:1]
        activations = self.activations_model.predict(validation_sample)
        #f = open('./modelTrain/activations_at_epoch_' + str(epoch) + '.npz', 'w')
        #np.savez(f, activations)
        #f.close()
        
        print(np.shape(activations[0]))
        print(np.shape(activations[1]))
        print(np.shape(activations[2]))
        print(np.shape(activations[3]))

In [ ]:
model2 = Model([_input], predictions)


callbacks_list = [
    #Interrupts training when improvement stops
    #Monitors the model’s validation accuracy
    #Interrupts training when accuracy has stopped improving for more than one epoch (that is, two epochs)
        keras.callbacks.EarlyStopping(
            monitor='acc',
            patience=1,
        ),
    #Saves the current weights after every epoch Path to the destination model file
    #These two arguments mean you won’t overwrite the model file unless val_loss has improved, which allows you to keep the best model seen during training.
        keras.callbacks.ModelCheckpoint(
            filepath='./modelTrain/model7.2.h2',
            monitor='val_loss',
            save_best_only=True,
        ),
    #Divides the learning rate by 10 when triggered The callback is triggered after the validation
    #loss has stopped improving for 10 epochs.
        keras.callbacks.ReduceLROnPlateau(#validation loss
            monitor='val_loss',
            factor=0.1,
            patience=10,
            
        ),
        ActivationLogger()


]
model2.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
model2.summary()

In [ ]:
history = model2.fit(x_train, y_train,
                    epochs=40,
                    batch_size=32,
                    callbacks=callbacks_list,
                    validation_split=0.2)

## 1D Convnet model <br>
illustrate tensorboard

In [ ]:
from keras import layers
from keras import Input
from keras.models import Model
from keras.layers import Flatten, Dense
from keras.optimizers import RMSprop

Embed_dim=128
_input = Input(shape=(None,))
Output_dim=32
Conv_windows=7
# We specify the maximum input length to our Embedding layer
# so we can later flatten the embedded inputs
x = layers.Embedding(max_features,Embed_dim, input_length=maxlen,name="embed")(_input)
x = layers.Conv1D(Output_dim,Conv_windows,activation="relu")(x)
x= layers.MaxPooling1D(5)(x)
x = layers.Conv1D(Output_dim,Conv_windows,activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
prediction = layers.Dense(1)(x)
model3 = Model(inputs=[_input],outputs=prediction)
model3.summary()

model3.compile(optimizer=RMSprop(lr=1e-4), # default learning rate is 10^-3, which is too fast, accuracy drops when iterating more steps
              loss='binary_crossentropy',
              metrics=['acc'])

In [ ]:
%%bash
mkdir -p ./model_log_dir

In [ ]:
np.shape(x_train)

In [4]:
from keras.models import load_model
model3 = load_model( filepath='./modelTrain/model7.2.Conv1d')

In [ ]:
callbacks = [
    keras.callbacks.TensorBoard(
        log_dir='./model_log_dir', #Log files will be written at this location.
        histogram_freq=1, #Records activation histograms every 1 epoch R
        embeddings_freq=1, #ecords embedding data every 1 epoch
    ),
    keras.callbacks.ModelCheckpoint(
            filepath='./modelTrain/model7.2.Conv1d',
            monitor='val_acc',
            save_best_only=True,
    )
]
history = model3.fit(x_train, y_train,
                    epochs=20,batch_size=128,
                    validation_split=0.2,
                    callbacks=callbacks)

Train on 20000 samples, validate on 5000 samples
Epoch 1/20
20000/20000 [==============================] - 151s 8ms/step - loss: 0.3148 - acc: 0.8587 - val_loss: 0.4142 - val_acc: 0.8270
Epoch 2/20
 1280/20000 [>.............................] - ETA: 2:07 - loss: 0.2902 - acc: 0.8789

In [ ]:
import tensorflow as tf
tf.keras.backend.clear_session()
